In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

import statsmodels as sms
import statsmodels.formula.api as smf

import seaborn as sns # for data visualization
sns.set_style("whitegrid")

from dateutil.relativedelta import *
from pandas.tseries.offsets import *

pd.set_option('display.max_columns', None)

DATA_FOLDER = '/Users/mmw/data/misp_data'

In [2]:
ret_decomp_dcf5 = pd.read_csv(f'{DATA_FOLDER}/decomp_dcf5y_hist_1960_2014.csv', index_col=0).replace([np.inf, -np.inf], np.nan).dropna()

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
ret_decomp_dcf5.head()

,vt,permno,ticker,me,be,year,jdate_comp,jdate_crsp,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_60,cumret_12,cumret_36,cumret_60,termlogret,termcumret,vt+3,me+3,be+3,vt+6,me+6,be+6,vt+12,me+12,be+12,vt+24,me+24,be+24,vt+36,me+36,be+36,vt+60,me+60,be+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp
71,10.08492,10001,EWST,10.612816,25293.0,2007,2006-06-30,2007-12-31,0.011433,0.160397,0.044941,0.011499,0.173977,0.045966,-0.230325,0.216031,0.262540,-0.205725,0.241141,0.300228,3.332619,27.011595,9.984871,10.570569,27152.0,9.984871,10.752607,27152.0,9.984871,10.477097,27152.0,10.209481,10.712601,30322.0,10.100357,11.319507,35907.0,9.838581,11.239867,77856.0,0.011433,-0.100049,-0.013197,0.056358,0.160397,-0.100049,-0.013197,0.233845,-0.230325,-0.100049,-0.013197,-0.034777,0.044941,0.124561,-0.013197,-0.024157,0.216031,0.015437,-0.013197,0.673973,0.262540,-0.246339,-0.013197,0.851556,0.527896
70,10.08492,10001,EWST,10.614634,25293.0,2007,2006-06-30,2007-11-30,0.066368,-0.003495,0.114277,0.068620,-0.003489,0.121063,-0.014518,0.418496,0.385518,-0.014414,0.519674,0.470376,3.332619,27.011595,9.984871,10.628062,27152.0,9.984871,10.766004,27152.0,9.984871,10.349001,27152.0,10.209481,10.566283,30322.0,10.100357,11.272806,35907.0,9.838581,11.260027,77856.0,0.066368,-0.100049,-0.013243,0.110641,-0.003495,-0.100049,-0.013243,0.245134,-0.014518,-0.100049,-0.013243,-0.161445,0.114277,0.124561,-0.013243,-0.168590,0.418496,0.015437,-0.013243,0.626666,0.385518,-0.246339,-0.013243,0.869439,0.529714
69,10.08492,10001,EWST,10.548344,25293.0,2007,2006-06-30,2007-10-31,0.031377,-0.007198,0.036080,0.031874,-0.007172,0.036739,-0.036607,0.345116,0.331830,-0.035945,0.412154,0.393516,3.332619,27.011595,9.984871,10.602865,27152.0,9.984871,10.529003,27152.0,9.984871,10.493957,27152.0,10.209481,10.550869,30322.0,10.100357,11.148880,35907.0,9.838581,11.311315,77856.0,0.031377,-0.100049,-0.011586,0.150707,-0.007198,-0.100049,-0.011586,0.078692,-0.036607,-0.100049,-0.011586,0.044521,0.036080,0.124561,-0.011586,-0.118985,0.345116,0.015437,-0.011586,0.570472,0.331830,-0.246339,-0.011586,0.984078,0.463423
68,10.08492,10001,EWST,10.589785,25293.0,2007,2006-06-30,2007-09-30,0.002456,0.013889,-0.008987,0.002459,0.013986,-0.008947,0.078308,0.238487,0.298470,0.081456,0.269327,0.347796,3.332619,27.011595,10.084920,10.612816,25293.0,9.984871,10.570569,27152.0,9.984871,10.517767,27152.0,10.209481,10.518818,30322.0,10.100357,11.120353,35907.0,9.838581,11.303199,77856.0,0.002456,0.000000,-0.012622,0.022455,0.013889,-0.100049,-0.012622,0.078813,0.078308,-0.100049,-0.012622,0.027331,-0.008987,0.124561,-0.012622,-0.190640,0.238487,0.015437,-0.012622,0.502252,0.298470,-0.246339,-0.012622,0.935759,0.504865
67,10.08492,10001,EWST,10.635131,25293.0,2007,2006-06-30,2007-08-31,-0.046761,0.019607,-0.033797,-0.045685,0.019800,-0.033232,0.125418,0.383630,0.338051,0.133622,0.467602,0.402212,3.332619,27.011595,10.084920,10.614634,25293.0,9.984871,10.628062,27152.0,9.984871,10.670236,27152.0,10.209481,10.511596,30322.0,10.100357,11.050621,35907.0,9.838581,11.307215,77856.0,-0.046761,0.000000,-0.013755,-0.019984,0.019607,-0.100049,-0.013755,0.090657,0.125418,-0.100049,-0.013755,0.131776,-0.033797,0.124561,-0.013755,-0.241893,0.383630,0.015437,-0.013755,0.390052,0.338051,-0.246339,-0.013755,0.895463,0.550211


In [6]:
ret_decomp_dcf5.groupby(pd.qcut(ret_decomp_dcf5.me, 10)).agg('mean')

,vt,permno,me,be,year,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_60,cumret_12,cumret_36,cumret_60,termlogret,termcumret,vt+3,me+3,be+3,vt+6,me+6,be+6,vt+12,me+12,be+12,vt+24,me+24,be+24,vt+36,me+36,be+36,vt+60,me+60,be+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp
me,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"(4.8069999999999995, 9.905]",8.600859,44191.410857,9.036391,7.115539e+04,1984.005864,0.035036,0.081059,0.358698,0.067318,0.150290,0.753098,0.174531,0.536661,0.860886,0.332017,1.258822,2.514272,1.986928,76.380914,8.655252,9.084849,7.216889e+04,8.709636,9.130462,7.343690e+04,8.816531,9.226319,7.637337e+04,8.965568,9.413720,8.403184e+04,9.064410,9.595585,9.307108e+04,9.118351,9.915652,1.134896e+05,0.035036,0.054394,-0.010888,-0.005787,0.081059,0.108778,-0.010888,-0.014339,0.174531,0.215673,-0.010888,-0.025101,0.358698,0.364709,-0.010888,0.012304,0.536661,0.463551,-0.010888,0.093252,0.860886,0.517493,-0.010888,0.352724,0.435533
"(9.905, 10.793]",9.682010,45589.275014,10.376746,1.328875e+05,1986.152454,0.033980,0.071030,0.288210,0.056335,0.119876,0.563452,0.144799,0.427725,0.700502,0.259770,0.902788,1.769517,2.761768,145.904316,9.716807,10.411811,1.359844e+05,9.752719,10.445661,1.387120e+05,9.823940,10.515795,1.462105e+05,9.927901,10.648661,1.620559e+05,10.001654,10.776751,1.792389e+05,10.070109,11.020933,2.223201e+05,0.033980,0.034796,-0.017368,0.000262,0.071030,0.070709,-0.017368,-0.001748,0.144799,0.141929,-0.017368,-0.002808,0.288210,0.245891,-0.017368,0.025374,0.427725,0.319643,-0.017368,0.078353,0.700502,0.388099,-0.017368,0.249686,0.694736
"(10.793, 11.479]",10.387793,46060.486728,11.147087,2.017268e+05,1986.658958,0.034217,0.068774,0.270680,0.053752,0.111498,0.498561,0.136882,0.403475,0.658565,0.233069,0.800965,1.551621,3.109150,228.262070,10.416025,11.179483,2.058259e+05,10.446195,11.212089,2.105855e+05,10.510515,11.276113,2.210198e+05,10.616409,11.399433,2.456252e+05,10.705960,11.524209,2.732452e+05,10.798882,11.757888,3.329026e+05,0.034217,0.028232,-0.018982,0.004060,0.068774,0.058402,-0.018982,0.006435,0.136882,0.122722,-0.018982,0.006146,0.270680,0.228616,-0.018982,0.023136,0.403475,0.318167,-0.018982,0.057481,0.658565,0.411089,-0.018982,0.194720,0.759294
"(11.479, 12.078]",10.977960,45605.085505,11.786733,3.054698e+05,1986.960145,0.032534,0.065034,0.245455,0.050460,0.102375,0.433248,0.126635,0.363332,0.602535,0.209112,0.681430,1.313444,3.460341,219.264222,11.004516,11.818348,3.123135e+05,11.031694,11.847896,3.191978e+05,11.085768,11.903118,3.333400e+05,11.184929,12.010985,3.686521e+05,11.273990,12.116314,4.057867e+05,11.387051,12.331187,5.004449e+05,0.032534,0.026556,-0.020219,0.004933,0.065034,0.053735,-0.020219,0.007243,0.126635,0.107808,-0.020219,0.008363,0.245455,0.206970,-0.020219,0.016850,0.363332,0.296030,-0.020219,0.032711,0.602535,0.409091,-0.020219,0.131979,0.808774
"(12.078, 12.649]",11.469884,45686.230859,12.361508,3.807610e+05,1988.435502,0.031381,0.060839,0.236559,0.048188,0.095609,0.405905,0.118641,0.355640,0.586828,0.192588,0.643288,1.240101,3.685465,262.472420,11.498454,12.390233,3.901526e+05,11.527120,12.417996,3.999016e+05,11.586319,12.471610,4.214394e+05,11.687457,12.578354,4.663562e+05,11.773010,12.687319,5.183590e+05,11.879638,12.893154,6.295925e+05,0.031381,0.028570,-0.022291,0.000151,0.060839,0.057236,-0.022291,-0.000729,0.118641,0.116434,-0.022291,-0.006174,0.236559,0.217573,-0.022291,-0.000708,0.355640,0.303125,-0.022291,0.022119,0.586828,0.409754,-0.022291,0.118845,0.891624
"(12.649, 13.202]",11.962873,45411.964259,12.926373,5.592718e+05,1989.517619,0.029775,0.057655,0.213034,0.045149,0.088573,0.359102,0.110330,0.323114,0.541193,0.175539,0.569336,1.089419,3.823474,292.769484,11.990204,12.952019,5.726562e+05,12.018429,12.976404,5.853863e+05,12.072880,13.022246,6.16887

In [11]:
ret_decomp_dcf5.groupby(pd.qcut(ret_decomp_dcf5.me, 10)).agg('std')

,vt,permno,me,be,year,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_60,cumret_12,cumret_36,cumret_60,termlogret,termcumret,vt+3,me+3,be+3,vt+6,me+6,be+6,vt+12,me+12,be+12,vt+24,me+24,be+24,vt+36,me+36,be+36,vt+60,me+60,be+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp,bm
me,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"(4.8069999999999995, 9.905]",1.433733,20405.404686,0.701974,3.631781e+05,10.432484,0.237430,0.329107,0.607025,0.299384,0.476435,1.573884,0.451719,0.713604,0.867668,0.808318,2.428574,4.341407,2.251245,433.820131,1.423678,0.729431,3.681466e+05,1.412817,0.757169,3.756609e+05,1.387440,0.806880,3.939218e+05,1.394478,0.895421,4.409490e+05,1.419720,0.971052,4.893981e+05,1.547878,1.085083,5.967324e+05,0.237430,0.294254,0.036197,0.371176,0.329107,0.406737,0.036197,0.514802,0.451719,0.548436,0.036197,0.711282,0.607025,0.736673,0.036197,0.945617,0.713604,0.880691,0.036197,1.108922,0.867668,1.177959,0.036197,1.336267,1.447869,165.696486
"(9.905, 10.793]",1.153554,22408.207192,0.252422,8.155030e+05,11.978015,0.201545,0.283645,0.546414,0.229606,0.367151,1.227705,0.400922,0.641177,0.786619,0.645629,1.631009,2.892039,2.237542,741.879254,1.158701,0.329711,8.370102e+05,1.162380,0.391612,8.546827e+05,1.167899,0.496195,9.123314e+05,1.202037,0.642634,1.032513e+06,1.247299,0.744778,1.162603e+06,1.357395,0.903038,1.474842e+06,0.201545,0.241332,0.028260,0.305716,0.283645,0.326557,0.028260,0.416761,0.400922,0.446914,0.028260,0.579587,0.546414,0.618183,0.028260,0.788529,0.641177,0.763690,0.028260,0.939616,0.786619,1.018369,0.028260,1.150364,1.130411,23.430347
"(10.793, 11.479]",1.054105,23228.208781,0.199024,1.328713e+06,12.353879,0.188166,0.265839,0.512266,0.216867,0.348799,0.917386,0.372908,0.598473,0.736746,0.554749,1.332096,2.444385,2.233614,1186.341196,1.062501,0.284995,1.352504e+06,1.067622,0.350091,1.382893e+06,1.073635,0.454370,1.447463e+06,1.106531,0.599852,1.630614e+06,1.136827,0.697185,1.823995e+06,1.254181,0.860647,2.074631e+06,0.188166,0.188188,0.025907,0.259896,0.265839,0.269949,0.025907,0.367562,0.372908,0.383539,0.025907,0.518089,0.512266,0.553804,0.025907,0.718990,0.598473,0.673197,0.025907,0.847775,0.736746,0.921551,0.025907,1.050619,1.036273,19.767996
"(11.479, 12.078]",1.003533,23844.409535,0.172579,1.591763e+06,12.455340,0.181624,0.252107,0.475953,0.202364,0.305005,0.781476,0.351330,0.556919,0.677133,0.483099,1.088586,2.013280,2.107216,785.920482,1.011093,0.256596,1.621276e+06,1.018828,0.317041,1.648073e+06,1.032902,0.413675,1.690483e+06,1.064111,0.548026,1.852315e+06,1.096375,0.642244,2.001935e+06,1.193502,0.789437,2.439913e+06,0.181624,0.170437,0.024837,0.243285,0.252107,0.237743,0.024837,0.336403,0.351330,0.328760,0.024837,0.465387,0.475953,0.481344,0.024837,0.643190,0.556919,0.611169,0.024837,0.779937,0.677133,0.862034,0.024837,0.991389,0.993482,12.330357
"(12.078, 12.649]",0.915622,23948.439019,0.165081,1.991465e+06,12.613786,0.177459,0.246425,0.455818,0.190629,0.288135,0.723028,0.338325,0.526932,0.650085,0.438116,1.020451,1.953486,2.115073,1009.098587,0.922246,0.251834,2.043496e+06,0.928233,0.313428,2.097047e+06,0.937022,0.402696,2.210630e+06,0.967978,0.524118,2.388717e+06,1.002692,0.611217,2.534137e+06,1.115916,0.747322,2.797331e+06,0.177459,0.163798,0.022622,0.239941,0.246425,0.230073,0.022622,0.332884,0.338325,0.320525,0.022622,0.455090,0.455818,0.462597,0.022622,0.610713,0.526932,0.593233,0.022622,0.732621,0.650085,0.835219,0.022622,0.920463,0.904876,8.710975
"(12.649, 13.202]",0.897486,24665.975782,0.158813,2.509331e+06,13.096762,0.170646,0.234896,0.434212,0.179463,0.260992,0.658808,0.322715,0.502368,0.614182,0.398397,0.916617,1.622063,2.026100,1265.783428,0.900355,0.244738,2.544511e+06,0.904705,0.303097,2.579004e+06,0.917678,0.393674,2.717987e+06,0.960795,0.512902,3.080725e+06,0.997109,0.591

In [9]:
ret_decomp_dcf5['bm'] = np.exp(np.log(ret_decomp_dcf5['be']) - ret_decomp_dcf5['me'])

In [10]:
ret_decomp_dcf5.groupby(pd.qcut(ret_decomp_dcf5.bm, 10)).agg('mean')

,vt,permno,me,be,year,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_60,cumret_12,cumret_36,cumret_60,termlogret,termcumret,vt+3,me+3,be+3,vt+6,me+6,be+6,vt+12,me+12,be+12,vt+24,me+24,be+24,vt+36,me+36,be+36,vt+60,me+60,be+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp,bm
bm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"(-0.000923, 0.221]",12.480838,49731.537710,14.069459,1.467793e+06,1994.668806,0.034175,0.048470,0.123964,0.056057,0.092360,0.308424,0.070025,0.201266,0.352738,0.161570,0.484451,0.891537,4.264443,613.011386,12.509752,14.082802,1.556389e+06,12.535753,14.096297,1.630395e+06,12.583877,14.114086,1.761485e+06,12.670128,14.160521,2.036779e+06,12.753263,14.223706,2.350680e+06,12.871287,14.339220,2.982014e+06,0.034175,0.028914,-0.039716,-0.015182,0.048470,0.054915,-0.039716,-0.027375,0.070025,0.103039,-0.039716,-0.056951,0.123964,0.189290,-0.039716,-0.095773,0.201266,0.272425,-0.039716,-0.115223,0.352738,0.390449,-0.039716,-0.117670,1.588621,0.143803
"(0.221, 0.33]",12.344357,48859.813922,13.727076,1.651027e+06,1993.503270,0.028904,0.044931,0.146397,0.046754,0.081011,0.309365,0.078850,0.231935,0.407997,0.154626,0.482234,0.906092,4.203540,528.158448,12.365310,13.743068,1.700658e+06,12.386687,13.759246,1.755717e+06,12.425578,13.791836,1.850361e+06,12.498194,13.852322,2.034101e+06,12.575478,13.927875,2.223277e+06,12.685402,14.074776,2.638003e+06,0.028904,0.020953,-0.034568,-0.004837,0.044931,0.042330,-0.034568,-0.009906,0.078850,0.081222,-0.034568,-0.016050,0.146397,0.153837,-0.034568,-0.027875,0.231935,0.231121,-0.034568,-0.029564,0.407997,0.341046,-0.034568,0.006488,1.382719,0.277593
"(0.33, 0.427]",12.131300,48153.772073,13.385793,1.590882e+06,1992.446588,0.029124,0.050463,0.176646,0.045422,0.084042,0.332505,0.089908,0.271020,0.455924,0.160326,0.521324,0.966212,4.009170,426.893775,12.154152,13.405536,1.635316e+06,12.175781,13.425618,1.674695e+06,12.217400,13.461615,1.751193e+06,12.292126,13.541257,1.929353e+06,12.362584,13.625896,2.113774e+06,12.468077,13.782658,2.546633e+06,0.029124,0.022852,-0.031362,-0.003031,0.050463,0.044481,-0.031362,-0.004540,0.089908,0.086101,-0.031362,-0.010022,0.176646,0.160826,-0.031362,-0.005228,0.271020,0.231284,-0.031362,0.008598,0.455924,0.336777,-0.031362,0.058586,1.254493,0.378354
"(0.427, 0.53]",11.855679,47328.976959,13.017748,1.173962e+06,1991.339458,0.028185,0.050089,0.191147,0.044288,0.082965,0.347054,0.096498,0.291415,0.485243,0.165978,0.551972,1.024149,3.784179,343.890554,11.877508,13.038563,1.203273e+06,11.901001,13.059234,1.232937e+06,11.946550,13.101118,1.295048e+06,12.025314,13.187318,1.425283e+06,12.096675,13.277551,1.579961e+06,12.201969,13.446220,1.959455e+06,0.028185,0.021829,-0.029052,-0.000988,0.050089,0.045323,-0.029052,-0.003741,0.096498,0.090872,-0.029052,-0.007314,0.191147,0.169635,-0.029052,-0.000063,0.291415,0.240996,-0.029052,0.018337,0.485243,0.346290,-0.029052,0.080127,1.162069,0.477940
"(0.53, 0.644]",11.705979,45895.736159,12.786138,1.302470e+06,1990.782744,0.026054,0.052034,0.203308,0.041641,0.084341,0.358346,0.100847,0.310983,0.522740,0.170293,0.573948,1.098016,3.732162,296.524459,11.727855,12.808758,1.332635e+06,11.751423,12.831748,1.369847e+06,11.799162,12.876021,1.428974e+06,11.881958,12.969160,1.560920e+06,11.958549,13.065559,1.705917e+06,12.072591,13.249450,2.011168e+06,0.026054,0.021876,-0.027004,0.000726,0.052034,0.045444,-0.027004,0.000162,0.100847,0.093183,-0.027004,-0.003217,0.203308,0.175979,-0.027004,0.006867,0.310983,0.252570,-0.027004,0.026180,0.522740,0.366612,-0.027004,0.094283,1.080158,0.585611
"(0.644, 0.777]",11.515664,44146.533858,12.514303,1.232923e+06,1989.875722,0.027746,0.054499,0.221890,0.043166,0.086725,0.378602,0.109290,0.336242,0.561569,0.178601,0.611514,1.170214,3.679530,258.760480,11.538085,12.538659,1.257178e+06,1

In [12]:
ret_decomp_dcf5.groupby(pd.qcut(ret_decomp_dcf5.bm, 10)).agg('std')

,vt,permno,me,be,year,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_60,cumret_12,cumret_36,cumret_60,termlogret,termcumret,vt+3,me+3,be+3,vt+6,me+6,be+6,vt+12,me+12,be+12,vt+24,me+24,be+24,vt+36,me+36,be+36,vt+60,me+60,be+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp,bm
bm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"(-0.000923, 0.221]",2.206425,26181.935781,2.220811,4.438051e+06,12.161555,0.201937,0.283305,0.540830,0.220435,0.325888,0.801514,0.399967,0.618081,0.742077,0.525194,1.107493,1.857142,2.544731,1636.683652,2.206387,2.225687,4.741343e+06,2.205502,2.232461,4.976391e+06,2.202132,2.245782,5.366266e+06,2.215838,2.266664,6.248357e+06,2.234883,2.279397,7.333501e+06,2.305684,2.309352,9.153481e+06,0.201937,0.159128,0.017507,0.265885,0.283305,0.216215,0.017507,0.362281,0.399967,0.299980,0.017507,0.497902,0.540830,0.446623,0.017507,0.677604,0.618081,0.585032,0.017507,0.812103,0.742077,0.830013,0.017507,0.989804,0.700277,0.054666
"(0.221, 0.33]",2.007821,24780.278844,1.977240,6.264341e+06,12.528418,0.183480,0.257337,0.488081,0.197388,0.291324,0.820880,0.358187,0.556343,0.667371,0.473827,1.052767,1.797249,2.363138,1806.176474,2.008512,1.982122,6.453340e+06,2.008573,1.986395,6.667425e+06,2.008606,1.996633,6.947483e+06,2.018230,2.015531,7.394092e+06,2.023796,2.024243,8.044053e+06,2.084743,2.052007,9.451211e+06,0.183480,0.139435,0.014546,0.231629,0.257337,0.201626,0.014546,0.322230,0.358187,0.283189,0.014546,0.437320,0.488081,0.444943,0.014546,0.612542,0.556343,0.564635,0.014546,0.716258,0.667371,0.818502,0.014546,0.891986,0.581857,0.031153
"(0.33, 0.427]",1.915005,24565.333803,1.869985,7.082633e+06,12.869864,0.174858,0.245204,0.464529,0.190128,0.294822,0.739045,0.340761,0.535432,0.645473,0.466819,1.014067,1.728005,2.249648,1554.814056,1.915543,1.874964,7.289624e+06,1.917215,1.880661,7.409247e+06,1.919982,1.894757,7.643579e+06,1.935695,1.914673,8.318794e+06,1.948122,1.930385,8.923885e+06,2.005169,1.966528,1.049694e+07,0.174858,0.154629,0.014769,0.230441,0.245204,0.213245,0.014769,0.317063,0.340761,0.295258,0.014769,0.438835,0.464529,0.456741,0.014769,0.612722,0.535432,0.587021,0.014769,0.726920,0.645473,0.835851,0.014769,0.908937,0.590755,0.028284
"(0.427, 0.53]",1.842837,24694.446912,1.794584,4.787159e+06,13.300836,0.173951,0.243176,0.456269,0.188974,0.278713,0.732944,0.334974,0.531037,0.639477,0.435894,1.076365,1.786040,2.217322,1436.262517,1.845176,1.800424,4.916517e+06,1.846888,1.805600,5.057487e+06,1.852466,1.816424,5.369556e+06,1.865404,1.843277,5.933485e+06,1.883574,1.860295,6.560020e+06,1.933010,1.898804,8.321962e+06,0.173951,0.157091,0.015203,0.229126,0.243176,0.225294,0.015203,0.322536,0.334974,0.313995,0.015203,0.443375,0.456269,0.464413,0.015203,0.618778,0.531037,0.612776,0.015203,0.748733,0.639477,0.854537,0.015203,0.938121,0.608111,0.029562
"(0.53, 0.644]",1.878936,24544.266008,1.826853,5.743410e+06,13.164175,0.172080,0.239129,0.446700,0.182967,0.278116,0.721497,0.330884,0.521195,0.634120,0.449237,1.029480,1.944619,2.189856,1227.671458,1.880853,1.832026,5.897970e+06,1.882696,1.837382,6.135091e+06,1.879563,1.846356,6.358595e+06,1.887645,1.871701,6.921696e+06,1.903511,1.890353,7.619165e+06,1.968242,1.923778,8.787881e+06,0.172080,0.168166,0.016086,0.236436,0.239129,0.237510,0.016086,0.328366,0.330884,0.338721,0.016086,0.462859,0.446700,0.495562,0.016086,0.640735,0.521195,0.629944,0.016086,0.768533,0.634120,0.863569,0.016086,0.956730,0.643430,0.032776
"(0.644, 0.777]",1.890490,24226.685996,1.845687,5.561000e+06,12.956055,0.169694,0.237264,0.437750,0.184719,0.287113,0.717703,0.326079,0.512258,0.620996,0.449515,1.087086,2.001417,2.169601,1028.801493,1.892510,1.852129,5.698283e+06,1.894331,1.856452,5.775151e+06,1.891196,1.862145,6.128447e+06,1.893506,1.875869,6.769872e+06,1.909979,1.894418,7.278342e+06,1.97186

In [13]:
def ols_coef(x,formula):
    return smf.ols(formula,data=x).fit().params

def fama_macbeth_summary(p):
    s = p.describe().T
    s['std_error'] = s['std']/np.sqrt(s['count'])
    s['tstat'] = s['mean']/s['std_error']
    return s[['mean','std_error','tstat', 'count', 'std']]

def fama_macbeth_reg_panel(regdf, xname='misp', yname='ret', 
                     csname='permno', tsname='jdate_crsp'): 
    # Panel factor setting FM reg
    gamma_cs = (regdf.groupby(tsname).apply(ols_coef,f'{yname} ~ {xname}'))
    gamma_cs = gamma_cs.rename(columns={"Intercept": "alpha_i_t", f"beta_{xname}": "lamba_t"})
    return fama_macbeth_summary(gamma_cs)

In [15]:
%%time
regdf = ret_decomp_dcf5[['jdate_crsp', 'permno', 'ticker', 'me', 'misp']].replace([np.inf, -np.inf], np.nan).dropna()
print(fama_macbeth_reg_panel(regdf, xname='me', yname='misp'))
print('')

               mean  std_error      tstat  count       std
alpha_i_t -0.366439   0.026344 -13.909764  600.0  0.645295
me         0.102951   0.001857  55.433813  600.0  0.045492

CPU times: user 3.7 s, sys: 57.8 ms, total: 3.75 s
Wall time: 3.75 s


In [17]:
%%time
regdf = ret_decomp_dcf5[['jdate_crsp', 'permno', 'ticker', 'bm', 'misp']].replace([np.inf, -np.inf], np.nan).dropna()
print(fama_macbeth_reg_panel(regdf, xname='bm', yname='misp'))
print('')

               mean  std_error      tstat  count       std
alpha_i_t  1.018320   0.012722  80.046566  600.0  0.311614
bm        -0.073359   0.006145 -11.937670  600.0  0.150526

CPU times: user 3.47 s, sys: 53.7 ms, total: 3.52 s
Wall time: 3.52 s
